In [8]:
import requests
from bs4 import BeautifulSoup
import re
import random
import pandas as pd
import time
from tqdm import tqdm     
import json
with open('./hidak_links.json', 'r') as f:
    links_list = json.load(f)

list1 = links_list['소아청소년과']

def scrape_hidoc_data(list1):
    head = { 'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"}
    
    data = []
    
    for link in tqdm(list1):
        url = f"https://mobile.hidoc.co.kr/healthqna/part/{link}"
        r = requests.get(url, headers=head)
        bs = BeautifulSoup(r.text, 'html.parser')

        date_data = bs.find("span", class_='date')
        date = date_data.text

        title_data = bs.select('.tit_qna')
        pattern = r'<strong class="tit_qna">\s*(.*?)\s*<div class="qna_info">'

        title_data_ = str(title_data[0])

        title = re.search(pattern, title_data_)

        if title:
            title_ = title.group(1)
        else:
            title_ = "매칭되는 부분이 없습니다."

        question_data = bs.find('div', class_='desc')   
        question = question_data.text.strip() 

        doctor_info = bs.find_all("strong", class_='link_doctor')

        doctor_list = []
        for x in range(len(doctor_info)):
            doctor_list.append(doctor_info[0].text)


        doc_hospital = bs.find_all("span", class_ ='txt_clinic')

        hospital_list = []
        for x in range(len(doc_hospital)):
            hospital_list.append(doc_hospital[0].text)

        a = bs.findAll("div", class_ = 'desc')
        answer = []
        for x in range(1,len(a)):
            answer.append(a[x].text.strip())

        data.append({
            'Date': date,
            'Title': title_,
            'Question': question,
            'Doctors': doctor_list,
            'Hospitals': hospital_list,
            'Answers': answer
        })
    
    df = pd.DataFrame(data)
    return df


# 함수 호출
df = scrape_hidoc_data(list1)


100%|██████████| 6469/6469 [36:17<00:00,  2.97it/s]  

            Date                                 Title  \
0     2020.07.13    아랫배가 아프면서 설사도 하고 윗배를 누르면 속이 울렁거립니다   
1     2017.05.26                          Mast 검사 결과수치   
2     2021.04.13                                 눈에 샴푸   
3     2018.04.14  제가 요즘 자꾸 친구들한테 짜증이나고,동생도 너무 싫고짜증납니다.   
4     2016.09.11                  편두통이 심한편인데 이유를 모르겠어요   
...          ...                                   ...   
6464  2021.02.13                      우울해요 제 상태가 어떤가요?   
6465  2022.02.20                 육류의 기름에만 설사를 심하게 합니다.   
6466  2016.10.04                편두통이 심한데 약을 많이 먹는게 ...   
6467  2017.12.14                      예방 접종 기간 문제에 대하여   
6468  2010.07.12                           종아리와 발이 부어요   

                                               Question             Doctors  \
0     안녕하세요 남자 중학생입니다. 토요일날 아침에 라면먹고 낮에 카라멜마끼야또를 먹고 ...  [홍인표 전문의, 홍인표 전문의]   
1     Ige토탈이 두가지네요Food ige total <100Total ige (cap)...           [서정호 전문의]   
2     머리 감다가 들어갔는데 이게 다 씻고 물방울이 튄 거예요샴푸가 혹시 다 안 씻겼을 ...           

In [26]:
df

,Date,Title,Question,Doctors,Hospitals,Answers
0,2020.07.13,아랫배가 아프면서 설사도 하고 윗배를 누르면 속이 울렁거립니다,안녕하세요 남자 중학생입니다. 토요일날 아침에 라면먹고 낮에 카라멜마끼야또를 먹고 ...,"[홍인표 전문의, 홍인표 전문의]","[닥터홍가정의학과의원, 닥터홍가정의학과의원]",[안녕하세요. 하이닥 가정의학과 상담의 홍인표입니다.많이 불편하시죠? 빨리 회복하세...
1,2017.05.26,Mast 검사 결과수치,Ige토탈이 두가지네요Food ige total <100Total ige (cap)...,[서정호 전문의],[연세한결소아청소년과의원 ],"[안녕하세요, 하이닥 소아청소년과 상담의 서정호 입니다.기계마다 검사 정상 수치는 ..."
2,2021.04.13,눈에 샴푸,머리 감다가 들어갔는데 이게 다 씻고 물방울이 튄 거예요샴푸가 혹시 다 안 씻겼을 ...,[한재병 전문의],[동래봉생병원],[안녕하세요. 하이닥 응급의학과 상담의 한재병입니다.샴푸가 눈 안에 들어가더라도 충...
3,2018.04.14,"제가 요즘 자꾸 친구들한테 짜증이나고,동생도 너무 싫고짜증납니다.","제가 요즘 자꾸 친구들한테 짜증이나고,동생도 너무 싫고짜증납니다.원래는 안 그러는데...",[고시환 전문의],[고시환의원],"[안녕하세요, 하이닥 소아청소년과 상담의 고시환 입니다.^^한 학생이 그러더군요.내..."
4,2016.09.11,편두통이 심한편인데 이유를 모르겠어요,편두통이 심한편인데 이유를 모르겠어요,[이지훈 전문의],[삼육서울병원],"[안녕하세요, 하이닥 신경과 상담의 이지훈 입니다. 상황에 대한 정확한 정보가 필요..."
...,...,...,...,...,...,...
6464,2021.02.13,우울해요 제 상태가 어떤가요?,"가끔식 눈물이 날때도 있고,나쁜생각과 안좋은 걱정을 많이하고요.모든게 하기가 싫고....",[홍인표 전문의],[닥터홍가정의학과의원],[안녕하세요. 하이닥 가정의학과 상담의 홍인표입니다.많이 힘드신 일이 있나요? 너무...
6465,2022.02.20,육류의 기름에만 설사를 심하게 합니다.,"위장장애 및 과민성대장, 장누수 증상까지 일으키고 있었는데요.1년동안 25kg 빠지...",[정성우 영양사],[],[안녕하세요. 하이닥 영양상담사 정성우입니다.단백질 섭취에 제한적이여서 많이 힘드실...
6466,2016.10.04,편두통이 심한데 약을 많이 먹는게 ...,편두통이 심한데 약을 많이 먹는게 좋지 않다는 말을 많이 들어서 약을 먹기 겁납니다...,[이지훈 전문의],[삼육서울병원],"[안녕하세요, 하이닥 신경과상담의 이지훈 입니다.1. 당연히 병원에 가셔야 합니다...."
6467,2017.12.14,예방 접종 기간 문제에 대하여,직업상 (후진국에서 고철수출입) 파상품에 노출될 위험이 높고 해외 여러곳을 돌아다니...,[강영록 전문의],[강의원],"[안녕하세요, 하이닥 소아청소년과 상담의 강영록 입니다.과거에는 어릴 때 DPT접종..."
